In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from timm.models import create_model
from tqdm import tqdm
import os

In [ ]:
# Step 1: Define the Dataset
class MultiLabelImageDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.image_col = "file"
        self.label_cols = self.data.columns[4:]  # assumes first 4 columns are metadata

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_path = row[self.image_col]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        labels = torch.tensor(row[self.label_cols].values.astype("float32"))
        return image, labels

print(" Dataset and class defined.")

 Dataset and class defined.


In [ ]:
# Step 2: Define Focal Loss
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = F.binary_cross_entropy(inputs, targets, reduction='none')
        pt = torch.where(targets == 1, inputs, 1 - inputs)
        loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss
        return loss.mean() if self.reduction == 'mean' else loss.sum()

print(" Focal loss ready.")


 Focal loss ready.


In [ ]:
# Step 3: Transforms + DataLoader
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_dataset = MultiLabelImageDataset('train.csv', transform=transform)
val_dataset = MultiLabelImageDataset('val.csv', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(" DataLoader is ready.")


 DataLoader is ready.


In [ ]:
# Step 4: Model + Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(" Using device:", device)

model = create_model("resnet50", pretrained=True, num_classes=19)
model.reset_classifier(num_classes=19)
model = nn.Sequential(model, nn.Sigmoid())
model.to(device)

criterion = FocalLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

print(" Model, optimizer, scheduler ready.")


✅ Using device: cuda


model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

✅ Model, optimizer, scheduler ready.
